# **Import Library**

In [1]:
import pandas as pd
import numpy as np

from datetime import date

# **Loading Data**

In [2]:

data = pd.read_csv("marketing_campaign.csv", sep = '\t')
display(data.shape, data.head(10))

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
5,7446,1967,Master,Together,62513.0,0,1,09-09-2013,16,520,...,6,0,0,0,0,0,0,3,11,0
6,965,1971,Graduation,Divorced,55635.0,0,1,13-11-2012,34,235,...,6,0,0,0,0,0,0,3,11,0
7,6177,1985,PhD,Married,33454.0,1,0,08-05-2013,32,76,...,8,0,0,0,0,0,0,3,11,0
8,4855,1974,PhD,Together,30351.0,1,0,06-06-2013,19,14,...,9,0,0,0,0,0,0,3,11,1
9,5899,1950,PhD,Together,5648.0,1,1,13-03-2014,68,28,...,20,1,0,0,0,0,0,3,11,0


In [3]:
data.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')

In [4]:
data_info = pd.DataFrame(columns=['Name of Col', 'Num of Null', 'Dtype', 'N_Unique'])

for i in range(0, len(data.columns)):
    data_info.loc[i] = (data.columns[i],
                        data[data.columns[i]].isnull().sum(),
                        data[data.columns[i]].dtypes,
                        data[data.columns[i]].nunique()) 
    
data_info

,Name of Col,Num of Null,Dtype,N_Unique
0,ID,0,int64,2240
1,Year_Birth,0,int64,59
2,Education,0,object,5
3,Marital_Status,0,object,8
4,Income,24,float64,1974
5,Kidhome,0,int64,3
6,Teenhome,0,int64,3
7,Dt_Customer,0,object,663
8,Recency,0,int64,100
9,MntWines,0,int64,776


# **Cleaning Data and Exploratory Data Analysis**

From the above,

1. "Income" has 24 null values.
   
2. "Dt_customer" needs to be parsed as DateTime.
   
3. "Education" and "Marital_Status" need to be encoded.

Because the number of null values in "Income" is just 24, I'd like to simply drop those rows.

**Drop the rows including null data**

In [5]:
data.dropna(inplace = True)

data.shape

(2216, 29)

**New feature creation**

In [6]:
data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'])
dates = []

for i in data['Dt_Customer']:
    i = i.date()
    dates.append(i)

print("The newest customer's enrowment date in the records : ", max(dates))
print("The oldest customer's enrowment date in the records : ", min(dates))

The newest customer's enrowment date in the records :  2014-12-06
The oldest customer's enrowment date in the records :  2012-01-08


Creating the new feature "Cus_for" to show the number of days the customers being the member of the shop.

In [7]:
days = []
d1 = max(dates)

for i in dates:
    delta = d1 - i
    days.append(delta)

days[0]

datetime.timedelta(days=971)

In [8]:
days_int = []
for i in range(0, len(days)):
    days_int.append(days[i].days) # with the method .days, we are able to get only the number of days from the timedelta type.

days_int[0]

971

In [9]:
data['Cus_for'] = days_int

print(data['Cus_for'].dtypes)
data['Cus_for'].head(1)

int64


Value counting the categorical variables

We will be exploring the unique category in the object features, "Marital_Status" and "Education".

- Marital_Status column has 8 categories.
- Education column has 5 categories.

In [10]:
print(data.Marital_Status.value_counts())
print(f'Num of unique category: {data.Marital_Status.nunique()}')
print('\n')
print(data.Education.value_counts())
print(f'Num of unique category: {data.Education.nunique()}')

Married     857
Together    573
Single      471
Divorced    232
Widow        76
Alone         3
Absurd        2
YOLO          2
Name: Marital_Status, dtype: int64
Num of unique category: 8


Graduation    1116
PhD            481
Master         365
2n Cycle       200
Basic           54
Name: Education, dtype: int64
Num of unique category: 5


New features creation
Ideation about creating new features and preprocessing the current features

* Year_Birth: We can create the age column
* Education: {Basic: Basic Education, Graduation: Graduated from Univ., Post graduate : (2n Cycle + Master + PhD)}
* Marital_Status: Changed the name of columns as 'Living_with', and mapped by {(Married + Together) : Partner, otherwise : Alone}
* Children = Kidhome + Teenhome
* Spent_All: adding all features staring Mnt
* PurchaseNumAll = all featrues staring Num
* PurDeal_PurAll_ratio: propotion of NumDealPurchases out of PurchaseNumAll
* Total_Promos = all featrues staring Accepted + Response
* Family_Size = ({"Alone": 1, "Partner":2})+ data["Children"]
* Is_Parent = (data.Children> 0, 1, 0)
* Drop Dt_Customer, ID, Year_Birth, Z_CostContact, Z_Revenue

In [11]:
data['Age'] = date.today().year - data['Year_Birth']

data['Education'].replace({
                            '2n Cycle' : 'Post Graduate',
                            'Master'   : 'Post Graduate',
                            'PhD'      : 'Post Graduate'}, inplace = True)

data['Children'] = data['Kidhome'] + data['Teenhome']
data['Spent_All'] = data['MntWines'] + data['MntFruits'] + data['MntMeatProducts'] + data['MntFishProducts'] + data['MntSweetProducts'] + data['MntGoldProds']
data['PurchaseNumAll'] = data['NumDealsPurchases'] + data['NumWebPurchases'] + data['NumCatalogPurchases'] + data['NumStorePurchases']
data['PurDeal_PurAll_ratio'] = data['NumDealsPurchases'] / data['PurchaseNumAll']
data['Total_Promos'] = data['AcceptedCmp1']+ data['AcceptedCmp2']+ data['AcceptedCmp3']+ data['AcceptedCmp4']+ data['AcceptedCmp5'] + data['Response']

data.rename(columns = {'Marital_Status' : 'Living_With'}, inplace = True)
data.loc[data['Living_With'] != 'Partner', 'Living_With'] = 'Alone'

data['Family_Size'] = data['Living_With'].replace({'Partner' : 2, 'Alone' : 1}) + data['Children']

data['Is_Parent'] = np.where(data['Children'] > 0, 1, 0)

display(data.shape, data.head(5))

,ID,Year_Birth,Education,Living_With,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,Response,Cus_for,Age,Children,Spent_All,PurchaseNumAll,PurDeal_PurAll_ratio,Total_Promos,Family_Size,Is_Parent
0,5524,1957,Graduation,Alone,58138.0,0,0,2012-04-09,58,635,...,1,971,64,0,1617,25,0.120000,1,1,0
1,2174,1954,Graduation,Alone,46344.0,1,1,2014-08-03,38,11,...,0,125,67,2,27,6,0.333333,0,3,1
2,4141,1965,Graduation,Alone,71613.0,0,0,2013-08-21,26,426,...,0,472,56,0,776,21,0.047619,0,1,0
3,6182,1984,Graduation,Alone,26646.0,1,0,2014-10-02,26,11,...,0,65,37,1,53,8,0.250000,0,2,1
4,5324,1981,Post Graduate,Alone,58293.0,1,0,2014-01-19,94,173,...,0,321,40,1,422,19,0.263158,0,2,1


In [12]:
data.describe()

,ID,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,Response,Cus_for,Age,Children,Spent_All,PurchaseNumAll,PurDeal_PurAll_ratio,Total_Promos,Family_Size,Is_Parent
count,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,...,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2212.000000,2216.000000,2216.000000,2216.000000
mean,5588.353339,1968.820397,52247.251354,0.441787,0.505415,49.012635,305.091606,26.356047,166.995939,37.637635,...,0.150271,512.006318,52.179603,0.947202,607.075361,14.880866,0.180577,0.448556,1.947202,0.714350
std,3249.376275,11.985554,25173.076661,0.536896,0.544181,28.948352,337.327920,39.793917,224.283273,54.752082,...,0.357417,232.469034,11.985554,0.749062,602.900476,7.670957,0.111044,0.892440,0.749062,0.451825
min,0.000000,1893.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,25.000000,0.000000,5.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,2814.750000,1959.000000,35303.000000,0.000000,0.000000,24.000000,24.000000,2.000000,16.000000,3.000000,...,0.000000,340.000000,44.000000,0.000000,69.000000,8.000000,0.076923,0.000000,1.000000,0.000000
50%,5458.500000,1970.000000,51381.500000,0.000000,0.000000,49.000000,174.500000,8.000000,68.000000,12.000000,...,0.000000,513.000000,51.000000,1.000000,396.500000,15.000000,0.166667,0.000000,2.000000,1.000000
75%,8421.750000,1977.000000,68522.000000,1.000000,1.000000,74.000000,505.000000,33.000000,232.250000,50.000000,...,0.000000,686.000000,62.000000,1.000000,1048.000000,21.000000,0.250000,1.000000,2.000000,1.000000
max,11191.000000,1996.000000,666666.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,...,1.000000,1063.000000,128.000000,3.000000,2525.000000,44.000000,1.000000,5.000000,4.000000,1.000000


There is 1893 in Year_Birth. Though this could be reasonable if that person is truly healthy but...

I think I should check the people whose age is more than 100 before dropping Year_Birth column.

In [13]:
data.loc[data['Age'] >= 90]

,ID,Year_Birth,Education,Living_With,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,Response,Cus_for,Age,Children,Spent_All,PurchaseNumAll,PurDeal_PurAll_ratio,Total_Promos,Family_Size,Is_Parent
192,7829,1900,Post Graduate,Alone,36640.0,1,0,2013-09-26,99,15,...,0,436,121,1,65,6,0.166667,0,2,1
239,11004,1893,Post Graduate,Alone,60182.0,0,1,2014-05-17,23,8,...,0,203,128,1,22,4,0.250000,0,2,1
339,1150,1899,Post Graduate,Alone,83532.0,0,0,2013-09-26,36,755,...,0,436,122,0,1853,15,0.066667,1,1,0


Can you believe the people more than 120 yr-old can generally earn the yearly income range 36,000 ~ 84,000?

Or

Can you believe the people more than 120 yr-old can actively purchase the items on the website?

I'd like to infer the age values in these 3 rows are, in my opinion, wrongly registered hence they should be dropped.

In [14]:
print(data.shape)

data.drop(index = [192, 239, 339], axis = 0, inplace = True)

print(data.shape)
data.Age.describe()

(2216, 38)
(2213, 38)


In [15]:
data['PurDeal_PurAll_ratio'].fillna(0, inplace = True)

I'll drop some more columns that are useless for further analysis.

* 'Dt_Customer': We will use 'Cus_for' column instead of it.
* 'Year_Birth': We will use 'Age' column instead of it.
* 'ID': useless
* 'Z_CostContact', 'Z_Revenue': Every row has the same value for these columns.
* 'Complain': Only 20 rows among 3213 rows has 1 and otherwise 0 hence it is also useless.

In [16]:
drop_cols = ['Dt_Customer', 'ID', 'Year_Birth', 'Z_CostContact', 'Z_Revenue', 'Complain']
data.drop(drop_cols, axis = 1, inplace = True)
print(data.shape)

(2213, 32)


In [20]:
data.to_csv('Marketing_Campaign.csv')